In [1]:
importall Base
using DataStructures

In [2]:
include("frames.jl")
include("spatial_inertia.jl")
include("rigid_body.jl")
include("spatial_motion_force.jl")
include("joint.jl")
include("tree.jl")
include("cache_element.jl")
include("frame_cache.jl")
include("mechanism.jl")

center_of_mass (generic function with 2 methods)

In [3]:
mechanism = Mechanism{Float64}("world");
world = root(mechanism)

RigidBody{Float64}(CartesianFrame3D("world"),Nullable{SpatialInertia{Float64}}())

In [4]:
J = rand(Mat{3, 3, Float64}); J = J * J'
c = rand(Vec{3, Float64})
m = rand()
inertia1 = SpatialInertia(CartesianFrame3D("body1"), J, c, m)
body1 = RigidBody(inertia1);
joint1 = Joint("1", QuaternionFloating());
joint1ToWorld = Transform3D(joint1.frameBefore, world.frame, nquatrand(), rand(Vec{3, Float64}))
body1ToJoint1 = Transform3D(inertia1.frame, joint1.frameAfter, nquatrand(), rand(Vec{3, Float64}))
attach!(mechanism, world, joint1, joint1ToWorld, body1, body1ToJoint1);

In [5]:
joint2 = Joint("2", Revolute(rand(Vec{3, Float64})));
inertia2 = SpatialInertia(joint2.frameAfter, J, c, m)
body2 = RigidBody(inertia2)
joint2ToBody1 = Transform3D(joint2.frameBefore, body1.frame, nquatrand(), rand(Vec{3, Float64}))
attach!(mechanism, body1, joint2, joint2ToBody1, body2);

In [6]:
x = MechanismState{Float64}(mechanism);

In [7]:
num_velocities(x)

7

In [8]:
configuration_vector(x)

8-element Array{Float64,1}:
 2.26467e-314
 2.2686e-314 
 2.26467e-314
 2.26467e-314
 2.26467e-314
 2.26467e-314
 2.26467e-314
 2.3034e-314 

In [9]:
zero_configuration!(joint1, x.q[joint1])

7-element Array{Float64,1}:
 1.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0

In [10]:
x.q

DataStructures.OrderedDict{Joint,Array{Float64,1}} with 2 entries:
  Joint("1",CartesianFram… => [1.0,0.0,0.0,0.0,0.0,0.0,0.0]
  Joint("2",CartesianFram… => [2.303401036e-314]

In [11]:
zero_configuration!(x)

In [12]:
cache = FrameCache(mechanism, x);

In [13]:
relative_transform(cache, joint2.frameAfter, body1.frame)

Transform3D{Float64}(CartesianFrame3D("after_2"),CartesianFrame3D("body1"),0.06880000269351419 + 0.6643778862981892im + 0.4380953054358256jm + 0.6016153980596022km,FixedSizeArrays.Vec{3,Float64}((0.18572969833561245,0.3053994904689805,0.9631195215820516)))

In [14]:
center_of_mass(mechanism, cache)

Point3D{Float64}(CartesianFrame3D("world"),FixedSizeArrays.Vec{3,Float64}((0.3534036068039646,1.3345302890861634,-1.5280298844622402)))

In [15]:
mass(mechanism)

1.5827602387776496

In [16]:
sum([isnull(body.inertia) ? 0. : get(body.inertia).mass for body in bodies(mechanism)])

1.5827602387776496

In [21]:
transform(cache, Point3D{Float64}(joint2.frameAfter), root(mechanism).frame)

Point3D{Float64}(CartesianFrame3D("world"),FixedSizeArrays.Vec{3,Float64}((0.5868542946486377,1.2549468941339792,-0.8213181620394061)))